# Twitter API

Obtain Tweets from Twitter API by using Tweepy (an easy-to-use Python Library)

to use Tweepy 
1. Make sure you have a twitter developer account (apps.twitter.com)
2. Install python and tweepy
3. Keywords or hashtag


More about it, go here http://www.tweepy.org/

Docs: https://tweepy.readthedocs.io/en/v3.5.0/

1) First install tweepy module using either:
"pip install tweepy", "pip install --user tweepy", python3 -m pip install tweepy (run python 3), easy_install tweepy


Once we have this, we make a Twitter application that will be used to interface with Python code we will write, and allow us to stream and process live tweets. After creating the Twitter application, we will leverage the "tweepy" module to stream the tweets.


Running a python file 

run: "python tweepy_streamer.py"
run: "python3 tweepy_streamer.py"

Wait for data to compile in tweets.txt




In [4]:
!pip3 install tweepy
!pip3 install twitter

In [6]:
# Importing Libraries

from tweepy import API
from tweepy import Cursor
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream

#import twitter_credentials
import numpy as np
import pandas as pd

In [7]:
# import data
#twitter = pd.read_csv('twitter_api/tweets.csv')
tweets = pd.read_csv('twitter_api/tweets.txt')
print(tweets)

Empty DataFrame
Columns: [[Status(_api=<tweepy.api.API object at 0x10b696190>,  _json={'created_at': 'Thu Mar 19 03:54:12 +0000 2020',  'id': 1240486705314095105,  'id_str': '1240486705314095105',  'text': 'RT @NathanLerner: Dear Republicans,  \n\nNext time you call climate change a hoax,  please remember the 2 months you spent calling the Coronavi…',  'truncated': False,  'entities': {'hashtags': [],  'symbols': [],  'user_mentions': [{'screen_name': 'NathanLerner',  'name': 'Nate Lerner',  'id': 755677202,  'id_str': '755677202',  'indices': [3,  16]}],  'urls': []},  'metadata': {'iso_language_code': 'en',  'result_type': 'recent'},  'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>',  'in_reply_to_status_id': None,  'in_reply_to_status_id_str': None,  'in_reply_to_user_id': None,  'in_reply_to_user_id_str': None,  'in_reply_to_screen_name': None,  'user': {'id': 2231845185,  'id_str': '2231845185',  'name': 'Pillsbury Doughpapi',  'screen

In [ ]:
# Extracting Data 

# Function to anaylze and categorize tweets        
df = pd.DataFrame(data=[tweet.text for tweet in tweets], columns=['Tweets'])

df['id'] = np.array([tweet.id for tweet in tweets])
#df['len'] = np.array([len(tweet.text) for tweet in tweets])
df['date'] = np.array([tweet.created_at for tweet in tweets])
df['source'] = np.array([tweet.source for tweet in tweets])
#df['likes'] = np.array([tweet.favorite_count for tweet in tweets])
#df['retweets'] = np.array([tweet.retweet_count for tweet in tweets])
df['location'] = np.array([tweet.place for tweet in tweets])
df['language'] = np.array([tweet.lang for tweet in tweets])

df

# External Resources

https://www.youtube.com/watch?v=wlnx-7cm4Gg